In [ ]:
renv::init()
renv::install("rvest")

In [ ]:
html <- rvest::read_html("https://quotes.toscrape.com/")

In [ ]:
quotes <- html |> 
  rvest::html_nodes(".text") |>
  rvest::html_text()

quotes

In [ ]:
file_conn <- file("./data/citacoes.txt", open = "a")

In [ ]:
url_pattern <- "https://quotes.toscrape.com/page/%d"
urls <- sprintf(url_pattern, 1:10)

header <- c("citacao", "autor")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for(url in urls) {
  html <- rvest::read_html(url)

  quotes <- html |>
    rvest::html_nodes(".text") |>
    rvest::html_text()


  authors <- html |>
    rvest::html_nodes(".author") |>
    rvest::html_text()

  writeLines(paste0(quotes, ";", authors, collapse = "\n"), file_conn, sep = "")
  Sys.sleep(1)
}

In [ ]:
close(file_conn)

In [ ]:
html <- rvest::read_html("https://quotes.toscrape.com/search.aspx")

hidden_inputs <- html |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

response <- httr::POST(
  url = "https://quotes.toscrape.com/filter.aspx",
  httr::add_headers(
    `Content-Type` = "application/x-www-form-urlencoded"
  ),
  body = list(
    author = "Haruki Murakami",
    tag = "thought",
    `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
  ),
  encode = "form"
)

In [ ]:
quotes <- response |>
  rvest::read_html() |>
  rvest::html_nodes(".quote .content") |>
  rvest::html_text()

quotes

In [ ]:
html <- rvest::read_html("https://quotes.toscrape.com/search.aspx")

authors <- html |>
  rvest::html_nodes("#author option") |>
  rvest::html_attr("value")

authors <- Filter(\(x) !is.na(x), authors)

hidden_inputs <- html |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

for(author in authors) {
  response <- httr::POST(
    url = "https://quotes.toscrape.com/filter.aspx",
    httr::add_headers(
      `Content-Type` = "application/x-www-form-urlencoded"
    ),
    body = list(
      author = author,
      `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
    ),
    encode = "form"
  )

  tags <- response |>
    rvest::read_html() |>
    rvest::html_nodes("#tag option")  |>
    rvest::html_attr("value")

  for(tag in tags) {
    response <- httr::POST(
      url = "https://quotes.toscrape.com/filter.aspx",
      httr::add_headers(
        `Content-Type` = "application/x-www-form-urlencoded"
      ),
      body = list(
        author = author,
        tag = tag,
        `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
      ),
      encode = "form"
    )

    quotes <- response |>
      rvest::read_html() |>
      rvest::html_nodes(".quote .content") |>
      rvest::html_text()

    quotes <- Filter(\(x) !identical(x, character(0)), quotes)
    cat(sprintf("Author: %s; Quotes %s; Tag: %s;\n", author, quotes, tag))
  }
}

In [ ]:
session <- rvest::session("https://quotes.toscrape.com/login")

hidden_inputs <- session |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

print(hidden_inputs)


In [ ]:
form <- session |> 
  rvest::html_form() |>
  (\(x) x[[1]])() |>
  rvest::html_form_set(
    username = "Username",
    password = "Password"
  )

session_2 <- rvest::session_submit(session, form)
session_2 |> rvest::session_jump_to("https://quotes.toscrape.com") |> rvest::read_html() |> rvest::html_text() |> cat()

In [70]:
login <- session |>
  rvest::html_form() |>
  (\(x) x[[1]])() |>
  rvest::html_form_set(
    username = "Username",
    password = "Password"
  )

logged_in <- session |> rvest::session_submit(login)